In [1]:

## Use to load the pdf and directory 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

## To convert entire corpus into chunks 
from langchain.text_splitter import RecursiveCharacterTextSplitter

# It is use for word embeddings here we are downloading 
## sentence-transformers/all-MiniLM-L6-v2 model 
from langchain.embeddings import HuggingFaceEmbeddings

# The pinecone takes all the chunks and embedding model and convert the corpus into
# vector embeddings and store it into pinecone on given index name 
## Instead of pinecone we can also use cromaDB
from langchain.vectorstores import Pinecone
import pinecone

## To use the quantize model because we are running it in cpu 
from langchain.llms import CTransformers

# use for creating the prompt in a perticular manner 
from langchain import PromptTemplate

#use to retrieve the answer to the given question
from langchain.chains import RetrievalQA

c:\Users\pouru\anaconda3\envs\fitness_chatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pinecone_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
pinecone_api_env = "xxxxxxxxxxxxxx"

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("C:/Users/pouru/All_about_pandas/Generative_AI/Fitness_chatbot/data")

In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 4669


In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [11]:
query_result = embeddings.embed_query("Hello world")
print("query result:", query_result)
print("Length", len(query_result))

query result: [-0.034477222710847855, 0.031023189425468445, 0.00673493230715394, 0.02610897831618786, -0.03936203941702843, -0.16030248999595642, 0.06692398339509964, -0.006441492587327957, -0.04745054617524147, 0.01475894171744585, 0.07087540626525879, 0.05552757903933525, 0.019193286076188087, -0.026251375675201416, -0.010109512135386467, -0.026940539479255676, 0.022307513281702995, -0.022226620465517044, -0.14969266951084137, -0.017493048682808876, 0.00767626753076911, 0.0543522946536541, 0.003254437353461981, 0.03172598034143448, -0.0846213847398758, -0.029406018555164337, 0.05159567669034004, 0.048124030232429504, -0.0033148014917969704, -0.05827920511364937, 0.04196932911872864, 0.022210698574781418, 0.12818889319896698, -0.022338908165693283, -0.011656295508146286, 0.06292832642793655, -0.03287626430392265, -0.09122607111930847, -0.03117542713880539, 0.052699606865644455, 0.0470348484814167, -0.08420299738645554, -0.030056146904826164, -0.020744740962982178, 0.009517806582152843

In [12]:
#Initializing the Pinecone
pinecone.init(api_key=pinecone_api_key,
              environment=pinecone_api_env)

index_name="fitness-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

KeyboardInterrupt: 

In [13]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [14]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [15]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-65ce19aa-4ca17da520a6c4982e2cbf20;979cbbcc-7a77-46b5-a2be-ff32e6189dac)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])